In [ ]:
pip install transformers

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import pandas as pd
from transformers import pipeline
import torch
# device = torch.device("cpu")
from transformers import GPTJForCausalLM, AutoTokenizer
from transformers import AutoModelForCausalLM, AutoTokenizer
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from transformers import AutoTokenizer,GPTJForCausalLM
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import T5Tokenizer, T5ForConditionalGeneration

from transformers import GPTNeoForCausalLM, GPT2Tokenizer

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large")
model = model.to(device)
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")

In [ ]:
df=pd.read_csv('PHEME_sentiment_vader.csv')

In [ ]:
df.head()

In [ ]:
df.drop(columns=['index','max_emotion','sentiment'],inplace=True)

In [ ]:
df.head()

In [ ]:
predictions = []
for index, row in df.iterrows():
    print(index)
    tweet = row['tweet']
    prompt = """
    You are given tweets regarding news events,you need to classify the tweets whether the given tweet is rumour or non-rumour.
    Tweet:charlie hebdo gunmen reportedly heard shouting prophet avenged + "emotion for this tweet is" + anger +"and sentiment for this tweet is"+ neutral
    Answer=rumour
    ###
    Tweet:11 confirmed dead francois hollande visit scene attack latest paris + "emotion for this tweet is" + sadness +"and sentiment for this tweet is"+ negative
    Answer=rumour
    ###
    Tweet:gunmen reportedly asked victims name shooting charliehebdo live updates + "emotion for this tweet is" + neutral +"and sentiment for this tweet is"+ negative
    Answer:rumour
    ###
    Tweet:awful 11 shot dead assault paris magazine charliehebdo + "emotion for this tweet is" + fear +"and sentiment for this tweet is"+ negative
    Answer:rumour
    ###
    Tweet:charlie hebdo journalist wandrille lanos tells france 2 men kalashnikovs entered office calls scene carnage + "emotion for this tweet is" + neutral +"and sentiment for this tweet is"+ neutral
    Answer:rumour
    ###
    Tweet:suspect paris kosher supermarket siege killed two people holding hostages + "emotion for this tweet is" + anger +"and sentiment for this tweet is"+ negative
    Answer:rumour
    ###
    Tweet:killed charlie hebdo suspects came firing security forces source afp + "emotion for this tweet is" + anger +"and sentiment for this tweet is"+ negative
    Answer:rumour
    ###
    Tweet:michael brown 17 yr old boy shot 10x amp killed police ferguson today media reports police shoot man blackboysonly + "emotion for this tweet is" + anger +"and sentiment for this tweet is"+ negative
    Answer:rumour
    ###
    Tweet:st louis unarmed teenager mikebrown shot ten times ferguson police officer accusation shoplifting + "emotion for this tweet is" + anger +"and sentiment for this tweet is"+ negative
    Answer:non-rumour
    ###
    Tweet:police narrative ferguson unarmed teen suspected unarmed robbery shot death weve still got problem chief + "emotion for this tweet is" + anger +"and sentiment for this tweet is"+ negative
    Answer:rumour
    ###
    Tweet: heres police report somehow ferguson cops valued box swishers nearly 50 time stop smokin guess + "emotion for this tweet is" + surprise +"and sentiment for this tweet is"+ positive
    Answer:rumour
    ###
    Tweet:breaking lawyer says browns friend confirms take cigarillos via msnbc ferguson + "emotion for this tweet is" + neutral +"and sentiment for this tweet is"+ positive
    Answer:rumour
    ###
    Tweet:ferguson pd beat amp charged innocent man property damage bleeding officers clothes youranonnews + "emotion for this tweet is" + sadness +"and sentiment for this tweet is"+ negative
    Answer:rumour
    ###
    Tweet:justify shooting unarmed teenager saying might committing misdemeanor fucking asshole ferguson + "emotion for this tweet is" + anger +"and sentiment for this tweet is"+ positive
    ###
    Tweet:germanwings plane crash airbus a320 crashes french alps 148 board latest updates + "emotion for this tweet is" + sadness +"and sentiment for this tweet is"+ negative
    Answer:rumour
    ###
    Tweet:reports moving body amidst germanwings wreckage a320 + "emotion for this tweet is" + surprise +"and sentiment for this tweet is"+ neutral
    Answer:rumour
    ###
    Tweet:ctvnews reporting second shooting behind peace tower parliament hill ottnews shooter loose + "emotion for this tweet is" + fear +"and sentiment for this tweet is"+ negative
    Answer:rumour
    ###
    Tweet:saw long gun heard one shot eye witness ottawashooting canadian parliament + "emotion for this tweet is" + surprise +"and sentiment for this tweet is"+ negative
    Answer:rumour
    ###
    Tweet:ottawa shootings suspect identified michael zehafbibeau 32 unclear shooters + "emotion for this tweet is" + neutral +"and sentiment for this tweet is"+ negative
    Answer:rumour
    ###
    Tweet:sydney opera house evacuated siege continuing local media reports 13 hostages+ "emotion for this tweet is" + fear +"and sentiment for this tweet is"+ neutral
    prediction:rumour
    ###
    Tweet:"""+tweet+""" Return your answer in rumour and non rumour only.
    Answer:"""
    inputs = tokenizer(prompt, return_tensors="pt")
    inputs = {k: v.to(device) for k, v in inputs.items()}
    outputs = model.generate(**inputs, max_length=900)
    response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    answer = response.split(":")[-1].strip()
    #predictions.append(answer)
    print(answer)
    if "non-rumour" in response.lower():
        prediction = "non-rumour"
    else:
        prediction = "rumour"
    predictions.append(prediction)

In [ ]:
df_predictions=pd.DataFrame(predictions,columns=['Prediction'])

In [ ]:
df_predictions.head()

In [ ]:
df_predictions.to_csv('Flan_T5_prediction_pheme_few_shot.csv')

In [ ]:
count=df_predictions[df_predictions['Prediction']=='rumour'].shape[0]

In [ ]:
count